In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.auto import tqdm
import lxml
import re #정규표현식 사용을 위한 라이브러리

data=pd.DataFrame(columns=['title','url','writer','date','content'])

def makeUrl(pg): # 입력한 페이지의 url을 변환하는 함수
    url='http://boannews.com/media/t_list.asp?Page='+str(pg)+'&kind='
    return url

def crawlingUrl(URL): # 한 페이지의 url을 읽어와 제목, url, 작성자를 반환하는 함수
    site=requests.get(URL)
    site_soup=BeautifulSoup(site.text, 'lxml')
    news_list=site_soup.select('div.news_list')
    for news in tqdm(news_list):
        news_title=news.select('span.news_txt')[0].text
        news_url=news.find('a').get('href')
        news_url='http://boannews.com'+news_url
        news_write=news.select('span.news_writer')[0].text
        news_writer=re.search(r'([\w\s]+) \|',news_write) #정규표현식으로 '\|' 앞에 해당하는 문자열 추출
        news_writer=news_writer.group(1) #괄호로 둘러싼 그룹에 접근해서 변수에 저장
        news_date=re.search(r'\| (.+)',news_write) #정규표현식으로 '\|' 뒤에 해당하는 문자열 추출
        news_date=news_date.group(1)

        content_site=requests.get(news_url) # 한 기사의 본문 url
        content_site_soup=BeautifulSoup(content_site.text, 'lxml')
        content=content_site_soup.find('div', id='news_content')
        if content:
            tag = content.find_all(['br','div','p','img'])
            for script in tag:
                script.extract()
            news_content = content.get_text('\n', strip=True)
            news_content = news_content.split('\n')
        else:
            news_content = []
        new_data=[news_title, news_url, news_writer,news_date, news_content]
        data.loc[len(data)]=new_data

n=int(input('페이지 수 입력: '))

for i in range(1,n+1):
    url=makeUrl(i)
    crawlingUrl(url)

data.to_csv('boannews.csv', encoding='utf-8-sig', index=False)

페이지 수 입력:  2


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]